# InfoGAN

Ref.:
CHEN, Xi et al.  
Infogan: Interpretable representation learning by information maximizing generative adversarial nets.  
In: Advances in neural information processing systems. 2016. p. 2172-2180.

![InfoGAN architecture](arch.png)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

Using TensorFlow backend.


In [3]:
# Generator

In [4]:
def build_generator(channels, latent_dim):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation("tanh"))

    gen_input = Input(shape=(latent_dim,))
    img = model(gen_input)

    model.summary()

    return Model(gen_input, img)

## Discriminator

In [5]:
def build_disk_and_q_net(img_shape, num_classes):

    img = Input(shape=img_shape)

    # Shared layers between discriminator and recognition network
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Flatten())

    img_embedding = model(img)

    # Discriminator
    validity = Dense(1, activation='sigmoid')(img_embedding)

    # Recognition
    q_net = Dense(128, activation='relu')(img_embedding)
    label = Dense(num_classes, activation='softmax')(q_net)

    # Return discriminator and recognition network
    return Model(img, validity), Model(img, label)

## save D, G

In [6]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [7]:
def save_model(G, D):
    save(G, "generator")
    save(D, "discriminator")

## Train

In [8]:
def mutual_info_loss(c, c_given_x):
    """The mutual information metric we aim to minimize"""
    eps = 1e-8
    conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
    entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

    return conditional_entropy + entropy

In [9]:
def sample_generator_input(num_classes, batch_size):
    # Generator inputs
    sampled_noise = np.random.normal(0, 1, (batch_size, 62))
    sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    sampled_labels = to_categorical(sampled_labels, num_classes=num_classes)

    return sampled_noise, sampled_labels

In [10]:
def sample_images(G, num_classes, epoch):
    r, c = 10, 10

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        sampled_noise, _ = sample_generator_input(num_classes, c)
        label = to_categorical(np.full(fill_value=i, shape=(r,1)), num_classes=num_classes)
        gen_input = np.concatenate((sampled_noise, label), axis=1)
        gen_imgs = G.predict(gen_input)
        gen_imgs = 0.5 * gen_imgs + 0.5
        for j in range(r):
            axs[j,i].imshow(gen_imgs[j,:,:,0], cmap='gray')
            axs[j,i].axis('off')
    fig.savefig("images/%d.png" % epoch)
    plt.close()

In [21]:
def train(D, G, combined,
          num_classes,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and categorical labels
        sampled_noise, sampled_labels = sample_generator_input(num_classes, batch_size)
        gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

        # Generate a half batch of new images
        gen_imgs = G.predict(gen_input)

        # Train on real and generated data
        d_loss_real = D.train_on_batch(imgs, valid)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)

        # Avg. loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator and Q-network
        # ---------------------

        g_loss = combined.train_on_batch(gen_input, [valid, sampled_labels])

        # Plot the progress
        print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, num_classes, epoch)

## main()

In [12]:
# create dirs
for d in ['images', 'saved_model']:
    if not os.path.exists(d):
        os.makedirs(d)

In [13]:
img_rows = 28
img_cols = 28
channels = 1
num_classes = 10

img_shape = (img_rows, img_cols, channels)
latent_dim = 72

In [14]:
optimizer = Adam(0.0002, 0.5)
losses = ['binary_crossentropy', mutual_info_loss]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
# Build and the discriminator and recognition network
D, auxilliary = build_disk_and_q_net(img_shape, num_classes)

D.compile(loss=['binary_crossentropy'],
          optimizer=optimizer,
          metrics=['accuracy'])

# Build and compile the recognition network Q
auxilliary.compile(loss=[mutual_info_loss],
                   optimizer=optimizer,
                   metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
# Build the generator
G = build_generator(channels, latent_dim)

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
gen_input = Input(shape=(latent_dim,))
img = G(gen_input)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              457856    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 128)      

In [18]:
# For the combined model we will only train the generator
D.trainable = False

# The discriminator takes generated image as input and determines validity
valid = D(img)
# The recognition network produces the label
target_label = auxilliary(img)

# The combined model  (stacked generator and discriminator)
combined = Model(gen_input, [valid, target_label])
combined.compile(loss=losses, optimizer=optimizer)

In [19]:
epochs=50000
epochs=5000

In [22]:
train(D, G, combined, num_classes,
      epochs=epochs, batch_size=128, sample_interval=50)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.16, acc.: 34.77%] [Q loss: 0.67] [G loss: 2.98]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 2.72, acc.: 50.00%] [Q loss: 0.70] [G loss: 3.06]
2 [D loss: 1.29, acc.: 52.73%] [Q loss: 0.99] [G loss: 2.96]
3 [D loss: 0.16, acc.: 93.75%] [Q loss: 0.62] [G loss: 2.69]
4 [D loss: 0.12, acc.: 96.48%] [Q loss: 0.20] [G loss: 2.83]
5 [D loss: 0.11, acc.: 97.27%] [Q loss: 0.08] [G loss: 2.70]
6 [D loss: 0.05, acc.: 99.22%] [Q loss: 0.02] [G loss: 2.58]
7 [D loss: 0.04, acc.: 99.22%] [Q loss: 0.01] [G loss: 2.68]
8 [D loss: 0.03, acc.: 99.61%] [Q loss: 0.01] [G loss: 2.56]
9 [D loss: 0.03, acc.: 99.61%] [Q loss: 0.02] [G loss: 2.56]
10 [D loss: 0.03, acc.: 99.61%] [Q loss: 0.00] [G loss: 2.73]
11 [D loss: 0.02, acc.: 99.61%] [Q loss: 0.00] [G loss: 2.69]
12 [D loss: 0.02, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.34]
13 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.57]
14 [D loss: 0.02, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.41]
15 [D loss: 0.02, acc.: 99.22%] [Q loss: 0.00] [G loss: 2.46]
16 [D loss: 0.03, acc.: 99.61%] [Q loss: 0.00] [G loss: 2.41]
17 [D loss: 0.